# Notebook de preparação do dataset de Acidentes

### Imports

In [ ]:
import pandas as pd
from resolve_path import ajuste_path

### Leitura do dataset de entrada pre-processado

In [ ]:
path = "data/"
path = ajuste_path(path)

pathUtil = path + "util/"

df_acidentes_padronizado = pd.read_csv(
    pathUtil + "acidentes/" + "BD_Acidentes_Padronizado.csv", encoding="utf-8", delimiter="|"
)

df_acidentes_padronizado.head(10)

### Ordenando acidentes por tempo (mais recente por último)

In [ ]:
# Renomeando temporariamente as colunas para 'day', 'month' e 'year'
df_acidentes_padronizado.rename(
    columns={'Dia': 'day', 'Mês': 'month', 'Ano': 'year'}, inplace=True)

# Criando coluna "Data" a partir das colunas 'day', 'month' e 'year'
df_acidentes_padronizado['Data'] = pd.to_datetime(
    df_acidentes_padronizado[['year', 'month', 'day']])

# Ordenando o dataframe pelas datas, do mais antigo para o mais recente
df_acidentes_padronizado = df_acidentes_padronizado.sort_values(by='Data')

# Renomeando as colunas de volta
df_acidentes_padronizado.rename(
    columns={'day': 'Dia', 'month': 'Mês', 'year': 'Ano'}, inplace=True)

# Reset do index
df_acidentes_padronizado = df_acidentes_padronizado.reset_index(drop=True)

In [ ]:
# Criando uma nova coluna "Indice acidente" com o valor do índice de cada linha
df_acidentes_padronizado['Indice acidente'] = [
    i for i in df_acidentes_padronizado.index]

### Removendo linhas cuja Classificação é de trajeto

In [ ]:
# contando quantas linhas possuem "trajeto" na coluna Classificação
conta_linhas_trajeto = (
    df_acidentes_padronizado["Classificação"].str.contains("trajeto").sum()
)

print(
    f"Quantidade de linhas que possuem 'trajeto' na coluna Classificação: {conta_linhas_trajeto} de {df_acidentes_padronizado.shape[0]} linhas totais."
)

# removendo as linhas que possuem "trajeto" na coluna Classificação
df_acidentes_padronizado = df_acidentes_padronizado[
    ~df_acidentes_padronizado["Classificação"].str.contains("trajeto")
]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

### Removendo linhas que não aconteceram na Eletrosul

In [ ]:
df_acidentes_padronizado = df_acidentes_padronizado[
    df_acidentes_padronizado["Empresa"] == "cgt eletrosul"
]

print(
    f"Quantidade de linhas após a remoção: {df_acidentes_padronizado.shape[0]}")

### Filtrando para apenas acidentes com empregado próprio

In [ ]:
print("Linhas antes da remoção de terceiros: ",
      df_acidentes_padronizado.shape[0])
qtd_linhas_com_terceiros = df_acidentes_padronizado[
    df_acidentes_padronizado["Empregado"] == "terceiro"
].shape[0]

print("Removendo " + str(qtd_linhas_com_terceiros) + " linhas com terceiros")

df_acidentes_padronizado = df_acidentes_padronizado[
    df_acidentes_padronizado['Empregado'] == 'proprio']

print("Linhas após a remoção de terceiros: ",
      df_acidentes_padronizado.shape[0])

### Corta string de local de instalação

In [ ]:
def local(linha):
    if isinstance(linha, str):
        string = str(linha)
        parts = string.split("-")

        if len(parts) > 3:
            result = "-".join(parts[:3])
        else:
            result = string

        return result
    return linha


df_acidentes_padronizado["Local de instalação"] = df_acidentes_padronizado["Local de instalação"].apply(
    local)

## Eliminação das linhas de acidentes irrelevantes e de predisposição física

In [ ]:
# Valores da coluna "Indice acidente" dos acidentes que foram julgados irrelevantes
acidentes_irrelevantes = [56, 102, 171, 206,
                          213, 294, 489, 667, 703, 788, 810, 844]

# Valores da coluna "Indice acidente" dos acidentes com causa relacionada à predisposição física do empregado
acidentes_por_predisposicao_fisica = [
    86, 179, 182, 205, 250, 257, 341, 443, 496]

# Eliminando acidentes_irrelevantes
df_acidentes_padronizado = df_acidentes_padronizado[~df_acidentes_padronizado["Indice acidente"].isin(
    acidentes_irrelevantes)]

# Eliminando acidentes_por_predisposicao_fisica
df_acidentes_padronizado = df_acidentes_padronizado[~df_acidentes_padronizado["Indice acidente"].isin(
    acidentes_por_predisposicao_fisica)]

### Salvando o novo dataset

In [ ]:
# Salva o dataset no folder modificados
df_acidentes_padronizado.to_csv(
    pathUtil + "acidentes/" + "BD_Acidentes_Preparado.csv",
    sep="|",
    encoding="utf-8",
    index=False,
)